In [1]:
import pandas as pd 
import numpy as np
import datetime
from pathlib import Path
import os

pd.set_option('display.max_columns', 500)


In [2]:
data_path = Path(os.getcwd()) / 'data' / 'WEC2022_Data'

In [3]:
data_path

WindowsPath('c:/wec_2022/wec_2022/data/WEC2022_Data')

In [4]:
def read_all_csv(data_path, data_type='train'):
    bkg_drop_cols = ['UPGRADED_FLAG', 'UPGRADE_TYPE', 'UPGRADE_SALES_DATE']
    bkg = pd.read_csv(data_path / f'BKG_{data_type}.csv', sep=';').drop(columns=bkg_drop_cols)
    tkt = pd.read_csv(data_path / f'TKT_{data_type}.csv', sep=';')
    fcp = pd.read_csv(data_path / f'FCP_{data_type}.csv', sep=';')
    df = fcp.merge(tkt, on='TICKET_NUMBER', how='left')
    df = df.merge(bkg, on='BOOKING_ID', how='left')
    return df 

In [5]:
df = read_all_csv(data_path=data_path)

In [6]:
# emd = pd.read_csv(data_path / 'EMD_train.csv', sep=';')

In [7]:
# df = df.merge(emd, how='left', left_on='TICKET_NUMBER', right_on='REFERENCE_TICKET_NUMBER')
# df.shape

In [8]:
# sus_air_type = (df[df['UPGRADED_FLAG'] == 'Y']['AIRCRAFT_TYPE'].value_counts() / df['AIRCRAFT_TYPE'].value_counts()).sort_values(ascending=False).head(5).index.tolist()

In [9]:
sus_air_type = ['763', '788', '789', '332', '787']

In [10]:
# sus_currency = (df[df['UPGRADED_FLAG'] == 1]['CURRENCY'].value_counts() / df['CURRENCY'].value_counts()).sort_values(ascending=False).head(6).index.tolist()

In [11]:
sus_currency = ['JPY', 'USD', 'CAD', 'SGD', 'VND', 'AED']

In [30]:
df[[col for col in df.columns if col not in drop_cols]]

,COUPON_NUMBER,TIME_DEPARTURE_LOCAL_TIME,FLIGHT_DISTANCE,FLIGHT_RANGE,BOOKED_CABIN,VAB,UPGRADED_FLAG,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,PAX_N,sale_to_flight_time,flight_len,if_additional_upgrade,same_carrier,is_sus_aircraft,is_sus_payment,intinerary_len
0,2,19.0,1344,SHORT-HAUL,Economy,STANDARD,0,620.35,1,ADULT,0,0,LOT TRAVEL,ROUND TRIP,121.0,4.0,0,0,1,125,14.500000,0,1,0,0,3
1,1,13.0,6942,LONG-HAUL,Economy,BASIC,0,2151.49,1,ADULT,0,0,AGENTS,ROUND TRIP,228.0,21.0,1,0,4,228,22.750000,0,1,1,0,3
2,2,16.0,7521,LONG-HAUL,Economy,BASIC,0,2270.18,0,ADULT,0,0,LOT.COM,ROUND TRIP,55.0,15.0,1,0,1,69,22.250000,1,1,1,1,3
3,1,6.0,721,SHORT-HAUL,Economy,STANDARD,0,1096.16,0,ADULT,0,0,AGENTS,ROUND TRIP,12.0,7.0,0,0,1,12,11.166667,0,1,0,0,5
4,2,18.0,522,SHORT-HAUL,Economy,OTHER,0,752.15,1,ADULT,0,0,AGENTS,ROUND TRIP,6.0,1.0,0,0,1,7,23.500000,0,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224582,1,19.0,523,SHORT-HAUL,Economy,STANDARD,0,813.64,1,ADULT,0,0,AGENTS,ONE WAY,0.0,None,0,0,1,0,4.666667,0,1,0,0,3
9224583,2,17.0,1070,SHORT-HAUL,Economy,SAVER,0,675.51,1,ADULT,0,0,LOT.COM,ROUND TRIP,116.0,8.0,0,0,3,124,8.000000,0,1,0,0,3
9224584,1,7.0,753,SHORT-HAUL,Economy,FLEX,0,1408.80,1,ADULT,0,0,AGENTS,ONE WAY,36.0,None,0,0,1,36,5.833333,0,1,0,0,3
9224585,1,18.0,940,SHORT-HAUL,Economy,SAVER,0,1157.97,1,ADULT,0,0,AGENTS,ROUND TRIP,37.0,3.0,0,0,3,37,7.416667,1,1,0,0,3


# Co zmieniamy 
1. 

In [35]:
def clean_df(data, data_type='train'):
    drop_cols = [
        "TICKET_NUMBER",
        "UPGRADE_SALES_DATE",
        "ORIGIN_AIRPORT_CODE",
        "DESTINATION_AIRPORT_CODE",
        "SALES_DATE",
        "FLIGHT_DATE_LOCAL",
        "MARKETING_CARRIER",
        "OPERATIONAL_CARRIER",
        "BOOKED_CLASS",
        "AIRCRAFT_TYPE",
        "FARE_BASIS",
        "UPGRADE_TYPE",
        "UPGRADE_SALES_DATE",
        "BOOKING_ID",
        "ORIGINAL_TICKET_NUMBER",
        "SEGMENTS",
        "FLIGHT_COUPONS",
        "FORM_OF_PAYMENT",
        "CURRENCY",
        "TOTAL_PRICE",
        "LOYAL_CUSTOMER_ID",
        "LOYAL_CUSTOMER_DATE_OF_BIRTH",
        "LOYAL_CUSTOMER_REGISTERED_DATE",
        "SALES_DATE",
        "SALES_MARKET",
        "SEGMENTS",
        "INTINERARY",
        "BOOKING_ORIGIN_AIRPORT",
        "BOOKING_ORIGIN_COUNTRY_CODE",
        "BOOKING_DEPARTURE_TIME_UTC",
        "BOOKING_DESTINATION_AIRPORT",
        "BOOKING_DESTINATION_COUNTRY_CODE",
        "BOOKING_ARRIVAL_TIME_UTC",
    ]
    
    sus_air_type = ["763", "788", "789", "332", "787"]
    sus_currency = ["JPY", "USD", "CAD", "SGD", "VND", "AED"]
    sus_payment = [
        "UNION",
        "CCDS6",
        "CCSW9",
        "NET R",
        "CCJC3",
        "CCAX3",
        "BARTE",
        "CCVI4",
        "PAY24",
    ]
    df = data.copy()
    df["FLIGHT_DATE_LOCAL"] = pd.to_datetime(df["FLIGHT_DATE_LOCAL"])
    df["SALES_DATE"] = pd.to_datetime(df["SALES_DATE"])
    df["sale_to_flight_time"] = df["FLIGHT_DATE_LOCAL"] - df["SALES_DATE"]
    df["sale_to_flight_time"] = df["sale_to_flight_time"].apply(lambda x: x.days)

    stay_lenght_map = {-9999: None}
    df["STAY_LENGTH_D"] = df["STAY_LENGTH_D"].replace(stay_lenght_map)
    df["BOOKING_ARRIVAL_TIME_UTC"] = pd.to_datetime(df["BOOKING_ARRIVAL_TIME_UTC"])
    df["BOOKING_DEPARTURE_TIME_UTC"] = pd.to_datetime(df["BOOKING_DEPARTURE_TIME_UTC"])

    df["flight_len"] = df["BOOKING_ARRIVAL_TIME_UTC"] - df["BOOKING_DEPARTURE_TIME_UTC"]
    df["flight_len"] = df["flight_len"].apply(lambda x: x.seconds / 3600)

    df["TIME_DEPARTURE_LOCAL_TIME"] = pd.to_datetime(df["TIME_DEPARTURE_LOCAL_TIME"])
    df["TIME_DEPARTURE_LOCAL_TIME"] = df["TIME_DEPARTURE_LOCAL_TIME"].apply(
        lambda x: x.hour
    )

    def get_if_add_upgrade(df):
        emd = pd.read_csv(data_path / "EMD_train.csv", sep=";")
        return np.where(
            np.isin(df["TICKET_NUMBER"], emd["REFERENCE_TICKET_NUMBER"].unique()), 1, 0
        )

    df["if_additional_upgrade"] = get_if_add_upgrade(df)
    df["same_carrier"] = np.where(
        df["MARKETING_CARRIER"] == df["OPERATIONAL_CARRIER"], 1, 0
    )
    df["is_sus_aircraft"] = np.where(np.isin(df["AIRCRAFT_TYPE"], sus_air_type), 1, 0)

    if data_type == 'train': 
        df["UPGRADED_FLAG"] = df["UPGRADED_FLAG"].map({"Y": 1, "N": 0})

    df["is_sus_payment"] = np.where(np.isin(df["FORM_OF_PAYMENT"], sus_payment), 1, 0)
    df["intinerary_len"] = df["INTINERARY"].apply(lambda x: len(x.split("-")))
    df["is_sus_currency"] = np.where(np.isin(df["FORM_OF_PAYMENT"], sus_currency), 1, 0)

    df["PAX_GENDER"] = df["PAX_GENDER"].map({"M": 1, "F": 0})

    df["CORPORATE_CONTRACT_FLG"] = df["CORPORATE_CONTRACT_FLG"].map({"Y": 1, "N": 0})
    df["LOYAL_CUSTOMER"] = df["LOYAL_CUSTOMER"].map({"Y": 1, "N": 0})

    df["BOOKING_LONG_HOUL_FLAG"] = df["BOOKING_LONG_HOUL_FLAG"].map({"Y": 1, "N": 0})
    df["BOOKING_DOMESTIC_FLAG"] = df["BOOKING_DOMESTIC_FLAG"].map({"Y": 1, "N": 0})

    return df.drop(columns=drop_cols)

In [22]:
# sus_payment = (df[df['UPGRADED_FLAG'] == 1]['FORM_OF_PAYMENT'].value_counts() / df['FORM_OF_PAYMENT'].value_counts()).sort_values(ascending=False).head(9).index.tolist()

In [50]:
final_df = clean_df(df)

MemoryError: Unable to allocate 1.31 GiB for an array with shape (19, 9224587) and data type int64

In [49]:
def oh_encoding(data): 
    df = data.copy()
    oh_cols = [
        "FLIGHT_RANGE",
        "BOOKED_CABIN",
        "VAB",
        "PAX_TYPE",
        "SALES_CHANNEL",
        "TRIP_TYPE",
    ]
    oh_df = pd.get_dummies(data[oh_cols], prefix=oh_cols, drop_first=True)
    return pd.concat([df, oh_df], axis=1)

In [48]:
final_df = oh_encoding(final_df)

MemoryError: Unable to allocate 5.50 GiB for an array with shape (40, 18449174) and data type float64

In [ ]:
final_df 

,COUPON_NUMBER,TIME_DEPARTURE_LOCAL_TIME,FLIGHT_DISTANCE,FLIGHT_RANGE,BOOKED_CABIN,VAB,UPGRADED_FLAG,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,PAX_N,sale_to_flight_time,flight_len,if_additional_upgrade,same_carrier,is_sus_aircraft,is_sus_payment,intinerary_len,is_sus_currency,FLIGHT_RANGE_LONG-HAUL,FLIGHT_RANGE_SHORT-HAUL,FLIGHT_RANGE_UNKNOWN,BOOKED_CABIN_Economy,BOOKED_CABIN_Premium,VAB_FLEX,VAB_FULL-FLEX,VAB_OTHER,VAB_SAVER,VAB_SEMI-FLEX,VAB_STANDARD,PAX_TYPE_CHILD,PAX_TYPE_INFANT,SALES_CHANNEL_ATO_CTO,SALES_CHANNEL_CALL CENTER,SALES_CHANNEL_DCS,SALES_CHANNEL_LOT TRAVEL,SALES_CHANNEL_LOT.COM,TRIP_TYPE_ONE WAY,TRIP_TYPE_ROUND TRIP
0,2.0,0.0,1344.0,SHORT-HAUL,Economy,STANDARD,NaN,620.35,NaN,ADULT,NaN,NaN,LOT TRAVEL,ROUND TRIP,121.0,4.0,NaN,NaN,1.0,125.0,14.500000,0.0,1.0,0.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.0,6942.0,LONG-HAUL,Economy,BASIC,NaN,2151.49,NaN,ADULT,NaN,NaN,AGENTS,ROUND TRIP,228.0,21.0,NaN,NaN,4.0,228.0,22.750000,0.0,1.0,1.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,0.0,7521.0,LONG-HAUL,Economy,BASIC,NaN,2270.18,NaN,ADULT,NaN,NaN,LOT.COM,ROUND TRIP,55.0,15.0,NaN,NaN,1.0,69.0,22.250000,1.0,1.0,1.0,1.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0.0,721.0,SHORT-HAUL,Economy,STANDARD,NaN,1096.16,NaN,ADULT,NaN,NaN,AGENTS,ROUND TRIP,12.0,7.0,NaN,NaN,1.0,12.0,11.166667,0.0,1.0,0.0,0.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,0.0,522.0,SHORT-HAUL,Economy,OTHER,NaN,752.15,NaN,ADULT,NaN,NaN,AGENTS,ROUND TRIP,6.0,1.0,NaN,NaN,1.0,7.0,23.500000,0.0,1.0,0.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9224583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
9224584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9224585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
